Einleitung

higher level api
part of spark-sql lib
build on top of rdd
 * in memory
 * partitioned
 * ready-only
 * resilient

Tabllenstruktur ähnlich relationalen Datenbanken

Spark wendet eine Reihe von Optimierungen an welche die Entwicklung verglichen mit RDD verkürzen und die Geschwindigkeit erhöhen.


## [Catalyst Optimizer](https://www.databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)

Query Optimizer to produce an optimized query plan

1. Analysis
2. Logical Optimization
3. Physical Planning
4. Code Generation

Let's have a look at a concrete Example

Nehmen wir an wir haben Code mit Dataframe API oder SQL.
Als erstes wird dieser Code analysiert:

### Analyse-Phase

* Syntax Check
* Unresolved logical Plan
  
  hier wird z.B. noch nicht geprüft ob Spalten existieren oder den richtigen Typ haben
* Nun wird eben diese mit Hilfe des Spark Catalogs geprüft und der Logische Plan erzeugt.

### Optimierung des logischen Planes

* Regelbasierte Optimierungen
  * [Predicate Pushdown](https://medium.com/microsoftazure/data-at-scale-learn-how-predicate-pushdown-will-save-you-money-7063b80878d7)
  * [Constant folding](https://en.wikipedia.org/wiki/Constant_folding)

Das ended im optimierten logischen Plan.

### Physical Plan

Für den optimierten logsichen Plan werden ein Reihe von Physischen Plänen berechnet und mit einem Kostenmodell bewertet. Der günstige wird dann genommen.

### Code Generation Phase

Am Ende sind es wieder RDDs die in Java Bytecode überführt werden.

### Beachte

all diese Optimierungen bekommst Du nicht wenn Du direkt RDD Code schreibst. Diese müsstest Du selber ausprogrammieren.
    



In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("catalyst").master("local[4]").getOrCreate()
spark

23/08/28 14:01:40 WARN Utils: Your hostname, pupil-a resolves to a loopback address: 127.0.1.1; using 167.235.141.210 instead (on interface eth0)
23/08/28 14:01:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/28 14:01:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/28 14:01:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [42]:
taxi_rides = spark.read.csv("YellowTaxis_202210.csv.gz", header = True, inferSchema=True)
taxi_rides_by_location = taxi_rides.groupby("PULocationID").count()
taxi_rides_81 = taxi_rides_by_location.where("PULocationID == 81")
taxi_rides_by_location.show()

+------------+-----+
|PULocationID|count|
+------------+-----+
|         148|38049|
|         243|  514|
|          31|   22|
|         137|43171|
|         251|    6|
|          85|   95|
|          65| 3042|
|         255| 1969|
|          53|   41|
|         133|   92|
|          78|  113|
|         155|  127|
|         108|   78|
|         211|26838|
|         193| 3383|
|          34|  113|
|         126|   67|
|         101|   50|
|         115|    5|
|          81|   73|
+------------+-----+
only showing top 20 rows



# Bild einfügen


## [Tungsten Engine](https://www.databricks.com/glossary/tungsten)

Improves efficiency of memory & CPU for spark applications during execution.

## Performance Vergleich

von der Datebricks Catalyst Seite einblenden

Learning: wenn Du die Dataframe API nutzt, dann ist es egal ob Du Python/Scala/SQL nutzt, der Code ist immer gleich und nahezu so schnell wie handoptimierter Code